In [1]:
# importing packages

from bokeh.io import output_notebook, output_file, show
from bokeh.core.properties import value
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge
output_notebook()

import time
import numpy as np
import pandas as pd
from gurobipy import *

Loading BokehJS ...

In [2]:
gurobi.version()

(9, 0, 0)

In [3]:
# base settings

L_shapes = True # True: allows both rectangles and L-shapes, False: only rectangles
grid = 3  # 1: one cell, 1: combined, 2: base, 3: divided
include_figures = False

In [4]:
# default settings

M = 1 * (grid == 1) + 3 * (grid == 2) + 6 * (grid == 3) + 6 * (grid == 4)  # number of rows
N = 1 * (grid == 1) + 3 * (grid == 2) + 9 * (grid == 3) + 18 * (grid == 4)  # number of columns
R = 4  # number of trades
step = 9999999

S_i = np.array(range(0, M + 1, 1))
S_j = np.array(range(0, N + 1, 1))
S_s = np.array(range(0, 4, 1))  # for the shape of L

b = [[]] * R
bb = [[]] * R


In [5]:
# data 

bb[0] = np.array([[0, 0, 0, 0, 0.8, 0, 0, 0, 0],
                  [0, 0, 2.5, 2.5, 3.8, 2.8, 2.8, 0, 0],
                  [0, 1.3, 2.3, 0, 4.1, 0, 2.1, 1.2, 0],
                  [0, 0, 2.2, 0, 0, 0, 2.0, 0, 0],
                  [0, 1.1, 2.1, 2.0, 2.0, 2.0, 2.1, 0.8, 0],
                  [0, 0, 0, 0, 1.0, 0, 0, 0, 0]])

bb[1] = np.array([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
                  [1.3, 0, 1.1, 0, 1.0, 0, 1.1, 0, 1.0],
                  [1.0, 0, 1.1, 3.0, 3.0, 3.0, 1.1, 0, 1.0],
                  [0.8, 0, 1.1, 3.0, 3.0, 3.0, 1.1, 0, 1.0],
                  [0.8, 0, 1.1, 0, 1.0, 0, 1.1, 0, 1.0],
                  [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]])

bb[2] = np.array([[0.2, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.2],
                  [0.2, 0.2, 1.8, 1.5, 0, 1.3, 1.5, 0, 0.2],
                  [0.2, 0, 2.8, 0, 0, 0, 2.5, 0.2, 0.2],
                  [0.2, 0.2, 3.8, 0, 8.2, 0, 3.5, 0, 0.2],
                  [0.2, 0, 4.8, 5.8, 6.8, 5.5, 4.5, 0.2, 0.2],
                  [0.2, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.2]])

bb[3] = np.array([[0, 0, 0, 0, 0, 0, 0, 2.0, 4.0],
                  [0, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 0],
                  [0, 1.2, 0, 0, 0, 0, 0, 1.2, 0],
                  [0, 1.2, 0, 0, 0, 0, 0, 1.2, 0],
                  [0, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [6]:
# adjust 

if grid == 1:
    for r in range(R):
        b[r] = np.array([[0.0]])
        for i in range(M):
            for j in range(N):
                b[r][i][j] = bb[r].sum()

elif grid == 2:
    for r in range(R):
        b[r] = np.array([[0.0, 0.0, 0.0]] * 3)
        for i in range(M):
            for j in range(N):
                b[r][i][j] = bb[r][i * 2:(i + 1) * 2, j * 3:(j + 1) * 3].sum()

elif grid == 3:
    b = bb

elif grid == 4:
    for r in range(R):
        b[r] = np.array([[0.0] * 18] * 6)
        for i in range(6):
            for j in range(18):
                b[r][i][j] = bb[r][i, j // 2] / 2

else:
    raise ValueError("grid should be either 1, 2, 3, or 4.")


In [7]:
# parameters

a = [[]] * R

for r in range(R):
    a[r] = np.zeros((M, N + 1))

for r in range(R):
    for i in range(M):
        for j in range(1, N + 1):
            a[r][i][j] = a[r][i][j - 1] + b[r][i][j - 1]


In [8]:
def draw_partitions(M, N, V, K):
    # drawing Matrix
    col = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0',
           '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8',
           '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

    m_num = 1 * (L_shapes == False) + 2 * (L_shapes == True)
    zone_text = " Zone " * (K == 1) +  " Zones " * (K > 1)
    plt = figure(x_range=(-0.01, N + 0.01), y_range=(-0.01, M + 0.01), 
                 plot_width=800, plot_height=round(800 * 6 / 9), toolbar_location="left")

    for k in range(K):
        if len(V[k]) == 4:
            plt.quad(top=[V[k][0]], bottom=[V[k][2]], left=[V[k][1]], right=[V[k][3]], color=col[k % 20], alpha=0.4)
        else:
            if V[k][-1] != 0:
                plt.quad(top=[V[k][0]], bottom=[V[k][4]], left=[V[k][5]], right=[V[k][3]], color=col[k % 20], alpha=0.4)
            if V[k][-1] != 1:
                plt.quad(top=[V[k][4]], bottom=[V[k][2]], left=[V[k][5]], right=[V[k][3]], color=col[k % 20], alpha=0.4)
            if V[k][-1] != 2:
                plt.quad(top=[V[k][4]], bottom=[V[k][2]], left=[V[k][1]], right=[V[k][5]], color=col[k % 20], alpha=0.4)
            if V[k][-1] != 3:
                plt.quad(top=[V[k][0]], bottom=[V[k][4]], left=[V[k][1]], right=[V[k][5]], color=col[k % 20], alpha=0.4)

    for i in range(0, M + 1):
        plt.line([0, N], [i, i], line_color='black', line_width=1, alpha=1)
    for i in range(0, N + 1):
        plt.line([i, i], [0, M], line_color='black', line_width=1, alpha=1)
        
    plt.xaxis.major_label_text_font_size = '0pt'
    plt.yaxis.major_label_text_font_size = '0pt'
    plt.xaxis.major_label_text_color = None
    plt.yaxis.major_label_text_color = None
    plt.xaxis.major_tick_line_color = None
    plt.xaxis.minor_tick_line_color = None
    plt.yaxis.major_tick_line_color = None
    plt.yaxis.minor_tick_line_color = None
    plt.xaxis.visible = False
    plt.yaxis.visible = False
    plt.xgrid.grid_line_color = None
    plt.ygrid.grid_line_color = None
    plt.title.align = 'center'
    plt.title.text_font_size = '15pt'
    
    name = "Figs/L" * (L_shapes == True) + "Rec" * (L_shapes == False) + "  --  Scn " + str(grid) + " -- Z " + str(
        K) + " -- map.html"
    
    
    plt.min_border_bottom = 6
    plt.min_border_top = 60
    plt.min_border_right = 2
    plt.min_border_left = 0

    
#     output_file(name)
    
    show(plt)
    return plt


In [9]:
def draw_chart(M, N, V, K):
    R_name = ['Mechanical  ', 'Framing  ', 'Electrical  ', 'Plumbing  ']

    col_rct = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0',
               '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8',
               '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

    col = ['#000075', '#42d4f4', '#f58231', '#8A2BE2', '#A52A2A', '#f032e6', '#4363d8', '#808000', 'green']

    takt = np.zeros(R * K).reshape(K, R)
    for i, v in enumerate(V):
        if len(v) == 4:
            for t in range(R):
                tmp = 0
                for row in range(v[0], v[2]):
                    tmp += a[t][row, v[3]] - a[t][row, v[1]]
                    takt[i][t] = tmp
        elif len(v) == 7:
            for t in range(R):
                tmp = 0
                for row in range(v[0], v[2]):
                    tmp += a[t][row, v[3]] - a[t][row, v[1]]
                if v[6] == 0:
                    for row in range(v[0], v[4]):
                        tmp -= a[t][row, v[3]] - a[t][row, v[5]]
                elif v[6] == 3:
                    for row in range(v[0], v[4]):
                        tmp -= a[t][row, v[5]] - a[t][row, v[1]]
                elif v[6] == 2:
                    for row in range(v[4], v[2]):
                        tmp -= a[t][row, v[5]] - a[t][row, v[1]]
                elif v[6] == 1:
                    for row in range(v[4], v[2]):
                        tmp -= a[t][row, v[3]] - a[t][row, v[5]]
                else:
                    print('***** Error q *****')
                takt[i][t] = tmp
        else:
            print('****** Error on len(v) ******')

    partitions = []
    for i in range(1, K + 1):
        partitions.append(str(i) + " zones (Z = " + str(i) + ")")

    data = {}
    data['prtitions'] = partitions
    for r in range(R):
        data[R_name[r]] = []
        for k in range(K):
            data[R_name[r]].append(takt[k, r])

    w = 2.8 / (5 * R - 1)
    source = ColumnDataSource(data=data)

    m_num = 'R' * (L_shapes == False) + 'L' * (L_shapes == True)
    zone_text = " Zone " * (K == 1) + " Zones " * (K > 1)
    p = figure(x_range=partitions, y_range=(0, BCP.ObjVal * 1.5), plot_height=700, plot_width=2500, tools="save",
               title= "            " + str(K) + zone_text + "  --  Model " + str(m_num) + "  --  Scenario " + str(grid),
               toolbar_location="left")
    
    
    title= "            " + "Model " + str(m_num) + '  --  ' + str(K) + zone_text +  "  --  Scenario " + str(grid)

    for k in range(K):
        p.quad(top=[BCP.ObjVal * 1.3], bottom=[0], left=[k], right=[k + 1], color=col_rct[k], alpha=0.2)

    for r in range(R):
        p.vbar(x=dodge('prtitions', w * 5 / 4 * r - 0.35 + w / 2, range=p.x_range), top=R_name[r], width=w,
               source=source,
               color=col[r], legend=value(R_name[r]))

    p.x_range.range_padding = 0.01
    p.xgrid.grid_line_color = None
    p.legend.location = "top_right"
    p.legend.orientation = "horizontal"
    p.legend.label_text_font_size = '36pt'
    p.yaxis.major_label_text_font_size = "36pt"

    p.xaxis.major_label_text_font_size = '0pt'
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.xaxis.minor_tick_line_color = None

    p.title.align = 'left'
    p.title.text_font_size = '42pt'
    
#     p.title.text_align = 'center'

    name = "Figs/L" * (L_shapes == True) + "Figs/Rec" * (L_shapes == False) + " -- Scn " + str(grid) + " -- Z " + str(
        K) + " -- bar.html"
    
    p.min_border_bottom = 15
        
#     output_file(name)
    show(p)
    return p


In [ ]:
cost = {}
for i1 in S_i[:-1]:
    for j1 in S_j[:-1]:
        for i2 in S_i[i1 + 1:]:
            for j2 in S_j[j1 + 1:]:
                tmp = np.zeros(R)
                for r in range(R):
                    for i in range(i1, i2):
                        tmp[r] += a[r][i, j2] - a[r][i, j1]
                cost[i1, j1, i2, j2] = tmp.max()

if L_shapes:
    for i1 in S_i[:-1]:
        for j1 in S_j[:-1]:
            for i2 in S_i[i1 + 1:]:
                for j2 in S_j[j1 + 1:]:
                    for i3 in S_i[i1 + 1:i2]:
                        for j3 in S_j[j1 + 1:j2]:
                            tmp = np.zeros(R)
                            for r in range(R):
                                for i in range(i1, i3):
                                    tmp[r] += a[r][i, j3] - a[r][i, j1]
                                for i in range(i3, i2):
                                    tmp[r] += a[r][i, j2] - a[r][i, j1]
                            cost[i1, j1, i2, j2, i3, j3, 0] = tmp.max()
    for i1 in S_i[:-1]:
        for j1 in S_j[:-1]:
            for i2 in S_i[i1 + 1:]:
                for j2 in S_j[j1 + 1:]:
                    for i3 in S_i[i1 + 1:i2]:
                        for j3 in S_j[j1 + 1:j2]:
                            tmp = np.zeros(R)
                            for r in range(R):
                                for i in range(i1, i3):
                                    tmp[r] += a[r][i, j2] - a[r][i, j1]
                                for i in range(i3, i2):
                                    tmp[r] += a[r][i, j3] - a[r][i, j1]
                            cost[i1, j1, i2, j2, i3, j3, 1] = tmp.max()
    for i1 in S_i[:-1]:
        for j1 in S_j[:-1]:
            for i2 in S_i[i1 + 1:]:
                for j2 in S_j[j1 + 1:]:
                    for i3 in S_i[i1 + 1:i2]:
                        for j3 in S_j[j1 + 1:j2]:
                            tmp = np.zeros(R)
                            for r in range(R):
                                for i in range(i1, i3):
                                    tmp[r] += a[r][i, j2] - a[r][i, j1]
                                for i in range(i3, i2):
                                    tmp[r] += a[r][i, j2] - a[r][i, j3]
                            cost[i1, j1, i2, j2, i3, j3, 2] = tmp.max()
    for i1 in S_i[:-1]:
        for j1 in S_j[:-1]:
            for i2 in S_i[i1 + 1:]:
                for j2 in S_j[j1 + 1:]:
                    for i3 in S_i[i1 + 1:i2]:
                        for j3 in S_j[j1 + 1:j2]:
                            tmp = np.zeros(R)
                            for r in range(R):
                                for i in range(i1, i3):
                                    tmp[r] += a[r][i, j2] - a[r][i, j3]
                                for i in range(i3, i2):
                                    tmp[r] += a[r][i, j2] - a[r][i, j1]
                            cost[i1, j1, i2, j2, i3, j3, 3] = tmp.max()


In [ ]:
# main

run_range = 2 * (grid == 1) + 10 * (grid == 2) + 16 * (grid == 3) + 16 * (grid == 4)

for K in range(1, run_range):  # number of partitions

    Si = np.array(range(0, M, 1))
    Sj = np.array(range(0, N + 1, 1))
    Sk = np.array(range(0, K + 1, 1))
    Sr = np.array(range(0, R, 1))

    TimeTotal = time.time()

    lb = 0
    for r in range(R):
        tmp = b[r].sum()
        if tmp > lb:
            lb = tmp
    lb = lb / K * (1 + step)
    cont = True
    it = 1

    # ------------------------------- 1
    # model
    BCP = Model('L shape model')
    # variables
    if L_shapes:
        x = BCP.addVars(S_i, S_j, S_i, S_j, S_i, S_j, S_s, lb=0.0, ub=1.0, vtype='B', name="X")
    xx = BCP.addVars(S_i, S_j, S_i, S_j, lb=0.0, ub=1.0, vtype='B', name="X")
    OBJ = BCP.addVar(lb=0.0, vtype='C', name="OBJECTIVE")
    BCP.update()

    # ------------------------------- 2
    # remove redundant variables
    for i1 in S_i:
        for j1 in S_j:
            for i2 in S_i[:i1 + 1]:
                for j2 in S_j:
                    BCP.remove(xx[i1, j1, i2, j2])
                    if L_shapes:
                        for i3 in S_i:
                            for j3 in S_j:
                                for s in S_s:
                                    BCP.remove(x[i1, j1, i2, j2, i3, j3, s])
    for i1 in S_i:
        for j1 in S_j:
            for i2 in S_i[i1 + 1:]:
                for j2 in S_j[:j1 + 1]:
                    BCP.remove(xx[i1, j1, i2, j2])
                    if L_shapes:
                        for i3 in S_i:
                            for j3 in S_j:
                                for s in S_s:
                                    BCP.remove(x[i1, j1, i2, j2, i3, j3, s])

    if L_shapes:
        for i1 in S_i:
            for j1 in S_j:
                for i2 in S_i[i1 + 1:]:
                    for j2 in S_j[j1 + 1:]:
                        for i3 in S_i[:i1 + 1]:
                            for j3 in S_j:
                                for s in S_s:
                                    BCP.remove(x[i1, j1, i2, j2, i3, j3, s])
        for i1 in S_i:
            for j1 in S_j:
                for i2 in S_i[i1 + 1:]:
                    for j2 in S_j[j1 + 1:]:
                        for i3 in S_i[i2:]:
                            for j3 in S_j:
                                for s in S_s:
                                    BCP.remove(x[i1, j1, i2, j2, i3, j3, s])
        for i1 in S_i:
            for j1 in S_j:
                for i2 in S_i[i1 + 1:]:
                    for j2 in S_j[j1 + 1:]:
                        for i3 in S_i[i1 + 1:i2]:
                            for j3 in S_j[:j1 + 1]:
                                for s in S_s:
                                    BCP.remove(x[i1, j1, i2, j2, i3, j3, s])
        for i1 in S_i:
            for j1 in S_j:
                for i2 in S_i[i1 + 1:]:
                    for j2 in S_j[j1 + 1:]:
                        for i3 in S_i[i1 + 1:i2]:
                            for j3 in S_j[j2:]:
                                for s in S_s:
                                    BCP.remove(x[i1, j1, i2, j2, i3, j3, s])

    BCP.update()

    # ------------------------------- 4
    # constraint:  enforce having K zones in total

    if L_shapes:
        BCP.addConstr((
                quicksum(x[i1, j1, i2, j2, i3, j3, s]
                         for i1 in S_i[:-1]
                         for j1 in S_j[:-1]
                         for i2 in S_i[i1 + 1:]
                         for j2 in S_j[j1 + 1:]
                         for i3 in S_i[i1 + 1:i2]
                         for j3 in S_j[j1 + 1:j2]
                         for s in S_s) +
                quicksum(xx[i1, j1, i2, j2]
                         for i1 in S_i[:-1]
                         for j1 in S_j[:-1]
                         for i2 in S_i[i1 + 1:]
                         for j2 in S_j[j1 + 1:]) == K),
            name="# of partitions")
    else:
        BCP.addConstr((
                quicksum(xx[i1, j1, i2, j2]
                         for i1 in S_i[:-1]
                         for j1 in S_j[:-1]
                         for i2 in S_i[i1 + 1:]
                         for j2 in S_j[j1 + 1:]) == K),
            name="# of partitions")
    BCP.update()

    # ------------------------------- 5
    # constraint:  enforce each cell be covered

    if L_shapes:
        BCP.addConstrs((
            quicksum(xx[i1, j1, i2, j2]
                     for i1 in S_i[:I + 1]
                     for j1 in S_j[:J + 1]
                     for i2 in S_i[I + 1:]
                     for j2 in S_j[J + 1:])
            + quicksum(x[i1, j1, i2, j2, i3, j3, s]
                       for i1 in S_i[:I + 1]
                       for j1 in S_j[:J + 1]
                       for i2 in S_i[I + 1:]
                       for j2 in S_j[J + 1:]
                       for i3 in S_i[i1 + 1: i2]
                       for j3 in S_j[j1 + 1: j2]
                       for s in S_s)
            - quicksum(x[i1, j1, i2, j2, i3, j3, 0]
                       for i1 in S_i[:I + 1]
                       for j1 in S_j[:J + 1]
                       for i2 in S_i[I + 1:]
                       for j2 in S_j[J + 1:]
                       for i3 in S_i[I + 1: i2]
                       for j3 in S_j[j1 + 1: J + 1])
            - quicksum(x[i1, j1, i2, j2, i3, j3, 1]
                       for i1 in S_i[:I + 1]
                       for j1 in S_j[:J + 1]
                       for i2 in S_i[I + 1:]
                       for j2 in S_j[J + 1:]
                       for i3 in S_i[i1 + 1: I + 1]
                       for j3 in S_j[j1 + 1: J + 1])
            - quicksum(x[i1, j1, i2, j2, i3, j3, 2]
                       for i1 in S_i[:I + 1]
                       for j1 in S_j[:J + 1]
                       for i2 in S_i[I + 1:]
                       for j2 in S_j[J + 1:]
                       for i3 in S_i[i1 + 1: I + 1]
                       for j3 in S_j[J + 1: j2])
            - quicksum(x[i1, j1, i2, j2, i3, j3, 3]
                       for i1 in S_i[:I + 1]
                       for j1 in S_j[:J + 1]
                       for i2 in S_i[I + 1:]
                       for j2 in S_j[J + 1:]
                       for i3 in S_i[I + 1: i2]
                       for j3 in S_j[J + 1: j2])
            == 1
            for I in S_i[:-1]
            for J in S_j[:-1]),
            name="single cell cover")
    else:
        BCP.addConstrs((
            quicksum(xx[i1, j1, i2, j2]
                     for i1 in S_i[:I + 1]
                     for j1 in S_j[:J + 1]
                     for i2 in S_i[I + 1:]
                     for j2 in S_j[J + 1:])
            == 1
            for I in S_i[:-1]
            for J in S_j[:-1]),
            name="single cell cover")
    BCP.update()

    # ------------------------------- 6
    # constraint Obj
    BCP.addConstrs((
        OBJ >= xx[i1, j1, i2, j2] * cost[i1, j1, i2, j2]
        for i1 in S_i
        for j1 in S_j
        for i2 in S_i[i1 + 1:]
        for j2 in S_j[j1 + 1:]),
        name="OBJ on Sum of the partition")

    if L_shapes:
        BCP.addConstrs((
            OBJ >= x[i1, j1, i2, j2, i3, j3, s] * cost[i1, j1, i2, j2, i3, j3, s]
            for i1 in S_i
            for j1 in S_j
            for i2 in S_i[i1 + 1:]
            for j2 in S_j[j1 + 1:]
            for i3 in S_i[i1 + 1:i2]
            for j3 in S_j[j1 + 1:j2]
            for s in S_s),
            name="OBJ on Sum of the partition L")

    BCP.update()

    #################################################################
    # -------- repetition
    
    while cont:
        # ------------------------------- 3
        lb_const1 = BCP.addConstrs((
            xx[i1, j1, i2, j2] == 0
            for i1 in S_i
            for j1 in S_j
            for i2 in S_i[i1 + 1:]
            for j2 in S_j[j1 + 1:]
            if cost[i1, j1, i2, j2] > lb),
            name="UB1")

        if L_shapes:
            lb_const2 = BCP.addConstrs((
                x[i1, j1, i2, j2, i3, j3, s] == 0
                for i1 in S_i
                for j1 in S_j
                for i2 in S_i[i1 + 1:]
                for j2 in S_j[j1 + 1:]
                for i3 in S_i[i1 + 1:i2]
                for j3 in S_j[j1 + 1:j2]
                for s in S_s
                if cost[i1, j1, i2, j2, i3, j3, s] > lb),
                name="UB2")

        BCP.update()

        # ------------------------------- 7
        BCP.setParam('OutputFlag', 0)
        # objective
        BCP.setObjective(OBJ, GRB.MINIMIZE)
        # solution
        BCP.optimize()

        # ------------------------------- closing    
        if BCP.Status == 2:
            cont = False
        else:
            lb = lb * (1 + step)
            BCP.remove(lb_const1)
            if L_shapes:
                BCP.remove(lb_const2)
            it += 1

    ttime = time.time() - TimeTotal
    print('{}\t{}\t\t{}'.format(K, round(BCP.ObjVal,4), round(ttime, 1)))

    #################################################################
    # -------- draw figure
    
    if include_figures:
        V = []
        for i1 in S_i[:-1]:
            for j1 in S_j[:-1]:
                for i2 in S_i[i1 + 1:]:
                    for j2 in S_j[j1 + 1:]:
                        if xx[i1, j1, i2, j2].X > 0.001:
                            V.append((i1, j1, i2, j2))
                        if L_shapes:
                            for i3 in S_i[i1 + 1:i2]:
                                for j3 in S_j[j1 + 1:j2]:
                                    for s in S_s:
                                        if x[i1, j1, i2, j2, i3, j3, s].X > 0.001:
                                            V.append((i1, j1, i2, j2, i3, j3, s))

        draw_partitions(M, N, V, K)
        draw_chart(M, N, V, K)
        print('\n--------------------------------------------------------------------------------------\n')


Using license file /global/home/users/ajabbari/gurobi.lic
Academic license - for non-commercial use only
1	70.1		22.6
